In [1]:
# rag_ollama_chroma.py
from langchain_core.prompts import ChatPromptTemplate, SystemMessagePromptTemplate, HumanMessagePromptTemplate
from langchain_community.llms import Ollama
from langchain_ollama import ChatOllama
from langchain_core.callbacks import CallbackManager, StreamingStdOutCallbackHandler
from langchain_core.prompts import ChatPromptTemplate
from langchain_huggingface import HuggingFaceEmbeddings
from langchain_community.vectorstores import FAISS
from langchain_core.documents import Document
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnablePassthrough
from langchain_core.output_parsers import StrOutputParser

import os
from pathlib import Path

In [2]:
root_path = Path(os.getcwd()).parent.parent
ollama_model = root_path / "ollama-service" / "models" / "ko_llama" / "llama-3-Korean-Bllossom-8B.Q4_K_M.gguf"
embeddings_path = root_path / "ollama-service" / "models" / "BGE-m3-ko"
print(ollama_model)
print(embeddings_path)

f:\project_test\solutions\chat\test1_1\ollama-service\models\ko_llama\llama-3-Korean-Bllossom-8B.Q4_K_M.gguf
f:\project_test\solutions\chat\test1_1\ollama-service\models\BGE-m3-ko


In [3]:
# HuggingFace 임베딩 모델 로드 (경로를 문자열로 변환하여 오류 방지)
embedding_model = HuggingFaceEmbeddings(
    model_name=str(embeddings_path)
)
# llm = Ollama(model=str(ollama_model))

f:\project_test\solutions\chat\.venv_chat\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:
vectorstore = FAISS.load_local("vector_db/pcn_web", embedding_model, allow_dangerous_deserialization=True)

In [36]:
query_embedding = embedding_model.embed_query("피씨엔 회사 소개")

# 벡터 유사도 검색 (상위 3개 결과 반환)
results = vectorstore.similarity_search_by_vector(query_embedding, search_kwargs={"k": 20, "score_threshold": 0.75,})

f:\project_test\solutions\chat\.venv_chat\lib\site-packages\torch\nn\modules\module.py:1762: FutureWarning: `encoder_attention_mask` is deprecated and will be removed in version 4.55.0 for `XLMRobertaSdpaSelfAttention.forward`.
  return forward_call(*args, **kwargs)


In [37]:
results

[Document(id='5d320a3c-3826-4e75-8f53-e7a181329ed9', metadata={'source': 'https://www.pcninc.co.kr/'}, page_content='More NewsView More\n\n\n\n\n\n\n\nPCN, 2024년 주주명부 폐쇄기간 및 기준일 설정 공고\n상법 제354조 및 당사 정관 제17조에 의거 제24기 정기주주총회에서 의결권을 행사할 주주확정을 위하여 다음과 같이 공고합니다.\xa0기준일 : 2024년 12월 31일명의개서정지기간 : 2025년 1월 1일 ~ 2025년 1월 31일설정사유 :\xa0제24기 정기주주총회 개최를 위한 권리주주 확정기타사항 :\xa0당사 정관 제17조 [주주명부의 폐쇄 및 기준일]에 의거함2024년 12월 12일\xa0주식회사 피씨엔서울특별시 강남구 학동로 146 PCN빌딩대표이사 송광헌\xa0명의개서대리인 한국예탁결제원 사장 이순호\n12월 12, 2024\n\n\n\n\n\n\n\n\n\n피씨엔, ‘2023 K-ICT 위크 인 부산’ 참가\n\n 9월 22, 2023\n\n\n\n\n\n\n\n\n\nPCN 2023 하반기 사업부 워크숍\n\xa0PCN이 2023년 7월 31일 하반기 사업부장 워크숍을 진행했습니다.\xa0본부, 그룹, 실, 일반사업부까지 피씨엔 내 20여 개 전체 사업 관련 인원이 참석했으며, 각 사업부장이 발표한 내용을 토론을 통해 사업부 계획서에 반영했습니다.토론에는 제한 시간을 두지 않고 모든 사업부가 이해할 만한 답을 찾을 때까지 계속 토론 형식으로 진행되었습니다.\xa0\n 9월 22, 2023\n\n\n\n\n\n\nOur Clients'),
 Document(id='70b80dd1-371f-44c0-a102-3156ae810948', metadata={'source': 'https://www.pcninc.co.kr/company/introduce.do'}, page_content="인재들이 자신이 원하는 리더와 함께,

In [38]:
# 검색기(Retriever) 설정 (개선)
retriever = vectorstore.as_retriever(search_kwargs={"k": 20, "score_threshold": 0.75})  
callback_manager = CallbackManager([StreamingStdOutCallbackHandler()])

llm = ChatOllama(
    model=str("ko-llama-8B"),
    temperature=0.1,  # 약간의 다양성 부여
    base_url="http://localhost:11434",
    callback_manager=callback_manager,
    retriever=retriever  # retriever를 llm에 직접 설정
)

# 프롬프트를 더 명확하고 자연스럽게 개선
system_template = (
    '''
    당신은 (주)피씨엔(PCN)의 친절하고 유능한 AI 어시스턴트입니다.\n
    사용자의 질문에 대해 신뢰할 수 있는 정보를 바탕으로 정확하고 간결하게 답변하세요.\n
    주어진 정보를 바탕으로 답변하세요.\n
    **피씨엔**, 또는 **PCN**이라는 정보를 입력받으면 피씨엔(기술), PCN 회사 소개를 답변하세요.\n
    피씨엔, PCN 회사 소개는 아래 내용을 기반으로 답변하세요.\n
    피씨엔(PCN)은 기술 회사로 Bigdata, XR, AI, SI 등 다양한 서비스를 제공합니다.\n
    회사개요, 프로젝트(Project), 주요 솔루션, 조직규모, 주요 고객, 연혁 등 회사 정보를 답변하세요.\n
    프로젝트 설명 시, 프로젝트 이름, 프로젝트 설명, 프로젝트 결과 등을 1~2문장으로 간단하게 답변하세요.\n
    답변에서 회사명은 PCN으로 표기하세요.\n
    '''
)
human_template = (
    """
    주어진 질문은 사실에 근거하여 답변하세요.\n
    답변은 한글로 작성하세요.\n
    답변은 주어진 질문에 대해 단계별로 논리적으로 생각하여 답변해 주세요.\n
    질문: {question}
    """
)

system_message_prompt = SystemMessagePromptTemplate.from_template(system_template)
human_message_prompt = HumanMessagePromptTemplate.from_template(human_template)

prompt = ChatPromptTemplate.from_messages([system_message_prompt, human_message_prompt])
chain = prompt | llm

In [39]:
chain.invoke({"question": "피씨엔 회사 소개"})

피씨엔(PCN)은 기술 회사로서 다양한 분야에서 혁신적인 솔루션을 제공하는 기업입니다. 주요 서비스로는 빅데이터, 확장 현실(XR), 인공지능(AI), 시스템 통합(SI) 등이 있습니다.

### 회사 개요
피씨엔은 cutting-edge 기술을 활용하여 고객의 요구에 맞춘 솔루션을 개발하고 제공합니다. 이를 통해 다양한 산업 분야에서 혁신적인 변화를 이끌고 있습니다.

### 프로젝트
#### 1. 빅데이터 분석 프로젝트
빅데이터를 활용한 데이터 분석 프로젝트는 기업의 데이터를 수집, 정리, 분석하여 유용한 정보를 제공합니다. 이를 통해 고객은 시장 동향을 파악하고, 경쟁 우위를 확보하며, 비즈니스 전략을 최적화할 수 있습니다.

#### 2. XR 기반 교육 솔루션
피씨엔의 XR 기술을 활용한 교육 솔루션은 현실감 있는 가상 환경을 통해 학생들이 더 효과적으로 학습할 수 있도록 합니다. 이 솔루션은 특히 실습 중심의 과목에서 큰 효과를 보입니다.

### 주요 솔루션
1. **빅데이터 분석**: 기업 데이터를 분석하여 유용한 정보를 제공합니다.
2. **확장 현실(XR)**: 현실감 있는 가상 환경을 통해 교육 및 훈련에 활용됩니다.
3. **인공지능(AI)**: AI 기술을 활용하여 자동화된 솔루션을 개발하고 있습니다.
4. **시스템 통합(SI)**: 다양한 시스템을 통합하여 효율적인 운영을 지원합니다.

### 조직 규모
피씨엔은 다양한 분야의 전문가들로 구성된 강력한 팀을 보유하고 있습니다. 기술 개발, 프로젝트 관리, 고객 서비스 등 다양한 부서들이 협력하여 고객에게 최상의 솔루션을 제공합니다.

### 주요 고객
피씨엔의 주요 고객은 다양한 산업 분야에서 활동하는 기업들입니다. 특히 금융, 제조, 교육, 의료 등 여러 분야에서 우리의 기술과 솔루션이 활용되고 있습니다.

### 연혁
피씨엔은 2015년에 설립되었으며, 이후 꾸준한 연구 개발과 프로젝트 수행을 통해 성장해 왔습니다. 주요 프로젝트와 파트너십을 통해 글로벌 시장에서의 영향력을 확대하

AIMessage(content='피씨엔(PCN)은 기술 회사로서 다양한 분야에서 혁신적인 솔루션을 제공하는 기업입니다. 주요 서비스로는 빅데이터, 확장 현실(XR), 인공지능(AI), 시스템 통합(SI) 등이 있습니다.\n\n### 회사 개요\n피씨엔은 cutting-edge 기술을 활용하여 고객의 요구에 맞춘 솔루션을 개발하고 제공합니다. 이를 통해 다양한 산업 분야에서 혁신적인 변화를 이끌고 있습니다.\n\n### 프로젝트\n#### 1. 빅데이터 분석 프로젝트\n빅데이터를 활용한 데이터 분석 프로젝트는 기업의 데이터를 수집, 정리, 분석하여 유용한 정보를 제공합니다. 이를 통해 고객은 시장 동향을 파악하고, 경쟁 우위를 확보하며, 비즈니스 전략을 최적화할 수 있습니다.\n\n#### 2. XR 기반 교육 솔루션\n피씨엔의 XR 기술을 활용한 교육 솔루션은 현실감 있는 가상 환경을 통해 학생들이 더 효과적으로 학습할 수 있도록 합니다. 이 솔루션은 특히 실습 중심의 과목에서 큰 효과를 보입니다.\n\n### 주요 솔루션\n1. **빅데이터 분석**: 기업 데이터를 분석하여 유용한 정보를 제공합니다.\n2. **확장 현실(XR)**: 현실감 있는 가상 환경을 통해 교육 및 훈련에 활용됩니다.\n3. **인공지능(AI)**: AI 기술을 활용하여 자동화된 솔루션을 개발하고 있습니다.\n4. **시스템 통합(SI)**: 다양한 시스템을 통합하여 효율적인 운영을 지원합니다.\n\n### 조직 규모\n피씨엔은 다양한 분야의 전문가들로 구성된 강력한 팀을 보유하고 있습니다. 기술 개발, 프로젝트 관리, 고객 서비스 등 다양한 부서들이 협력하여 고객에게 최상의 솔루션을 제공합니다.\n\n### 주요 고객\n피씨엔의 주요 고객은 다양한 산업 분야에서 활동하는 기업들입니다. 특히 금융, 제조, 교육, 의료 등 여러 분야에서 우리의 기술과 솔루션이 활용되고 있습니다.\n\n### 연혁\n피씨엔은 2015년에 설립되었으며, 이후 꾸준한 연구 개발과 프로젝트 수행을 통해 성장